In [22]:
from lxml import html
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd 
import numpy as np
import pylab as plt
%matplotlib inline
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
df_lacks = pd.DataFrame()

In [4]:
columns = ['Number', 'Player', 'Age', 'Position', 'Mins played', 'Appearances', 'Line Ups', 'Sub In', 'Sub Out', 'Sub On Bench', 'Goals', 'Asists', 'Yellow', 'Yellow 2nd', 'Red', 'Club', 'League', 'Country']

In [5]:
def createList(url):
    page = requests.get(url) #,proxies=proxyDict)
    s = re.search('submenu_dropdown(.*)yui-main', str(page.content)).group(1)
    soup = BeautifulSoup(s, 'html.parser')
    period_tags = soup.find_all(re.compile("(option)")) #, class_=re.compile("(number|position|name|shirtnumber)"))
    country = [str(i).split('/')[2] for i in period_tags]
    team = [str(i).split('/')[3] for i in period_tags]
   
    return  [i for i in zip(country, team)]

In [6]:
def compute(country, league, team):
    page = requests.get('http://int.soccerway.com/teams/'+country+'/'+team+'/squad/') #,proxies=proxyDict)
    s = re.search('table squad sortable(.*)Coach', str(page.content)).group(1)
    soup = BeautifulSoup(s, 'html.parser')
    period_tags = soup.find_all(re.compile("(td)"), class_=re.compile("(number|position|name|shirtnumber)"))
    list = [i.get_text() for i in period_tags] # if 'coach' not in str(i)]
    #del list[-1:]
    A = np.array(list)
    B = pd.DataFrame(np.reshape(A, (int(A.size/15), 15)))
    B['15'] = team.replace('-', ' ').upper()
    B['16'] = league.replace('-', ' ').upper()
    B['17'] = country.replace('-', ' ').upper()
    return B

In [7]:
def concat(clubs, league):
    df = compute(clubs[0][0], league, clubs[0][1])
    for team in range(1,len(clubs)):
        df = pd.concat([df, compute(clubs[team][0], league, clubs[team][1])])
    return df

In [25]:
def type_changer(df):
    columns = [2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
    for num in columns:
        if '' in df[num].unique():
            df = df[df[num] != '']
        df[num] = df[num].astype(int)
    return df

In [9]:
def column_changer(df):
    df.columns = columns
    return df

# England

#### Premier League

In [26]:
df_premier_league = column_changer(type_changer(concat(\
                    createList('http://int.soccerway.com/teams/england/chelsea-football-club/661/squad/'), 'PREMIER LEAGUE')))

#### Championship

In [11]:
df_championship = column_changer(type_changer(concat(\
                  createList('http://int.soccerway.com/teams/england/brighton--hove-albion-fc/703/squad/'), 'CHAMPIONSHIP')))

#### League One

In [12]:
df_league_one = column_changer(type_changer(concat(\
                createList('http://int.soccerway.com/teams/england/afc-wimbledon/4593/squad/'), 'LEAGUE ONE')))

#### League Two

In [13]:
df_league_two = column_changer(type_changer(\
                concat(createList('http://int.soccerway.com/teams/england/plymouth-argyle-fc/701/'), 'LEAGUE TWO')))

In [14]:
df_england = pd.concat([df_premier_league,df_championship,df_league_one,df_league_two])

In [15]:
len(df_england['Club'].unique())

92

# Germany

#### Bundesliga

#### Bundesliga 2

# Merge and Export

In [18]:
#pd.concat([df_premier_league,df_bundesliga]).to_csv('premier_league.csv')